In [ ]:
# Caricamento dati necessari
#
#
tweets = "./iorestoacasa_1.json"

# Creazione dataFrame
df_Tweets = spark.read.format("json").option("inferSchema", "true").option("multiLine", "true").load(tweets)
df_Tweets.printSchema()

In [ ]:
#df_Tweets.select("extended_tweet.full_text").show()
#df_Tweets.select("extended_tweet.full_text", "truncated").show(truncate = 15)
df = df_Tweets.select("extended_tweet.full_text").where("truncated = true")
#df.show(truncate = 140)

df_noRetweet = df_Tweets.where(df_Tweets["retweeted_status"].isNull())
df_Tweet_OrgFull = df_noRetweet\
    .select("extended_tweet.full_text")\
    .where("truncated = true")
df_Tweet_OrgNotFull = df_noRetweet\
    .select("text")\
    .where("truncated = false")

df_withRetweet = df_Tweets.where(df_Tweets["retweeted_status"].isNotNull())
df_Tweet_RetFull = df_withRetweet\
    .select("retweeted_status.extended_tweet.full_text")\
    .where("retweeted_status.truncated = true")
df_Tweet_RetNotFull = df_withRetweet\
    .select("retweeted_status.text")\
    .where("retweeted_status.truncated = false")

df_Tweet_OrgFull = df_Tweet_OrgFull.withColumnRenamed("full_text","text")
df_Tweet_RetFull = df_Tweet_RetFull.withColumnRenamed("full_text","text")

#df_Tweet_OrgFull.printSchema()
#df_Tweet_OrgNotFull.printSchema()
#df_Tweet_RetFull.printSchema()
#df_Tweet_RetNotFull.printSchema()

df_Tweet = df_Tweet_OrgFull\
    .union(df_Tweet_OrgNotFull)\
    .union(df_Tweet_RetFull)\
    .union(df_Tweet_RetNotFull)

In [ ]:
df_Tweet.count()
df_Tweet = df_Tweet.dropDuplicates()
df_Tweet.count()